In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
os.getcwd()

'/root/disaster-tweets/model'

In [4]:
import pandas as pd
import numpy as np

In [5]:
# import nltk
# nltk.download('punkt')

In [6]:
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize
tt = TweetTokenizer()

In [7]:
training_data = pd.read_csv('/root/disaster-tweets/data/train.csv')
testing_data = pd.read_csv('/root/disaster-tweets/data/test.csv')

In [8]:
training_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
training_data.text[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [10]:
training_data.text[training_data.target==0].values[0:2]

array(["What's up man?", 'I love fruits'], dtype=object)

In [11]:
training_data.shape

(7613, 5)

In [12]:
training_data[training_data.target==0].shape[0]

4342

In [13]:
training_data[training_data.target==1].shape[0]

3271

In [14]:
training_data['keyword'].unique()

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [15]:
training_data['location'].unique()

array([nan, 'Birmingham', 'Est. September 2012 - Bristol', ...,
       'Vancouver, Canada', 'London ', 'Lincoln'], dtype=object)

In [16]:
def replace_nan(dataframe):
    nan_replaced=dataframe.fillna('')
    return nan_replaced

def decode_encoded_values(dataframe):
    string_columns=training_data.columns[training_data.dtypes==object]
    for col in string_columns:
        dataframe[col] = dataframe[col].str.replace(r'%20', ' ')
    return dataframe

def clean_data(dataframe):
    nan_replaced=replace_nan(dataframe)
    decoded_df=decode_encoded_values(nan_replaced)
    return decoded_df


In [17]:
def concatenate_to_text_column(decoded_df, column_names):
    combination_name= 'combined_text'
    decoded_df[combination_name]=decoded_df['text'].str.cat(decoded_df[column_names].astype(str), sep=", ")
    return decoded_df

def tokenised_tweets(decoded_df):
    decoded_df['tokens']=decoded_df.apply(lambda row: tt.tokenize(row['combined_text']), axis=1)
    return decoded_df

def preprocess_tweets(decoded_df, column_names):
    concatenated_df=concatenate_to_text_column(decoded_df, column_names)
    tokenised_df=tokenised_tweets(concatenated_df)
    sentences = tokenised_df['combined_text'].to_numpy()
    if 'target' in tokenised_df.columns:
        tokenised_df['target'] = tokenised_df.target.to_numpy()
    return tokenised_df


In [18]:
new_training_data=clean_data(training_data)
new_testing_data=clean_data(testing_data)

In [19]:
#model 1: combining keywords, location and text 

In [20]:
preprocessed_training=preprocess_tweets(new_training_data, ['keyword', 'location'])
preprocessed_testing=preprocess_tweets(new_testing_data, ['keyword', 'location'])

In [21]:
preprocessed_training.head()

,id,keyword,location,text,target,combined_text,tokens
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,Our Deeds are the Reason of this #earthquake M...,"[Our, Deeds, are, the, Reason, of, this, #eart..."
1,4,,,Forest fire near La Ronge Sask. Canada,1,"Forest fire near La Ronge Sask. Canada, ,","[Forest, fire, near, La, Ronge, Sask, ., Canad..."
2,5,,,All residents asked to 'shelter in place' are ...,1,All residents asked to 'shelter in place' are ...,"[All, residents, asked, to, ', shelter, in, pl..."
3,6,,,"13,000 people receive #wildfires evacuation or...",1,"13,000 people receive #wildfires evacuation or...","[13,000, people, receive, #wildfires, evacuati..."
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,Just got sent this photo from Ruby #Alaska as ...,"[Just, got, sent, this, photo, from, Ruby, #Al..."


In [22]:
from sklearn.model_selection import train_test_split

train_tokens, test_tokens, train_labels, test_labels = train_test_split(preprocessed_training.tokens,
                                                                              preprocessed_training.target,
                                                                              test_size=0.1)
len(train_tokens), len(test_tokens), len(train_labels), len(test_labels)

(6851, 762, 6851, 762)

In [23]:
# pip install tensorflow

In [24]:
# pip install tensorflow_hub

In [25]:
import tensorflow as tf
import tensorflow_hub as hub

    
universal_sentence_encoder = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder-large/5',
                                           input_shape=[], # shape of inputs coming to our mod
                                           dtype=tf.string, # data type of inputs coming to th
                                           trainable=False, # keep the pretrained weights (we'
                                           name="USE")
    


2023-10-17 19:33:16.306914: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 19:33:17.060337: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-17 19:33:17.060444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-17 19:33:17.064293: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-17 19:33:17.513773: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 19:33:17.516947: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [26]:
tf.keras.utils.set_random_seed(4)

model = tf.keras.models.Sequential([
#     text_vectorizer,
    universal_sentence_encoder,
#     tf.keras.layers.Input(shape=(6851,), dtype=tf.string),
#     bert,
#     tf.keras.layers.Dense(units=1024, activation="relu"),
#     tf.keras.layers.Dropout(rate=0.5),
#     tf.keras.layers.Dense(units=512, activation="relu"),
#     tf.keras.layers.Dropout(rate=0.5),
#     tf.keras.layers.Dense(units=256, activation="relu"),
#     tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=32, activation="relu"),
#     tf.keras.layers.Dropout(rate=0.5),
    tf.keras.layers.Dense(units=1, activation="sigmoid")
], name="universal_sentence_encoder")

# Compile the model 
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
             metrics=["accuracy"])

In [27]:
history = model.fit(x=train_tokens,
                   y=train_labels,
                   epochs=50,
                   validation_data=(test_tokens, test_labels),
                   validation_steps=len(test_tokens),
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                               patience=10,
                                                               restore_best_weights=True),
                             tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                                  factor=0.2,
                                                                  patience=2,
                                                                  min_lr=1e-8)
                             ])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [ ]:
#model 2: combining keywords and text 

In [ ]:
#model 3: combining location and text 

In [ ]:
#model 4: Just text 